<a href="https://colab.research.google.com/github/klsharma22/TensorflowCertifcationCourse/blob/main/06_Transfer_Learning_in_Tensorflow_Part_3_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from helper_functions import unzip_data, make_confusion_matrix, walk_through_dir
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

In [4]:
TRAIN_DIR = '/Users/klsharma22/Desktop/TensorflowCertifcationCourse/Transfer Learning/101_food_classes_10_percent/train'
TEST_DIR = '/Users/klsharma22/Desktop/TensorflowCertifcationCourse/Transfer Learning/101_food_classes_10_percent/test'
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
LABEL_MODE = 'categorical'

In [8]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(TRAIN_DIR, label_mode= CLASS_MODE, batch_size= BATCH_SIZE, image_size= IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(TEST_DIR, label_mode= CLASS_MODE, batch_size= BATCH_SIZE, image_size= IMG_SIZE)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [14]:
data_augmentation = Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomHeight(0.2),
    layers.RandomWidth(0.2),
    layers.RandomZoom(0.2),
    layers.Rescaling(1/255.)
], name= 'data_augmentation')

In [17]:
base_model = tf.keras.applications.MobileNet(include_top= False)
base_model.trainable = False

inputs = layers.Input(shape= IMG_SIZE + (3, ), name= 'input_layer')
x = data_augmentation(inputs)
x = base_model(x, training= False)
x = layers.GlobalAveragePooling2D(name= 'global_average_pooling_layer')(x)
outputs = layers.Dense(len(train_data.class_names), activation= 'softmax', name= 'output_layer')(x)

model = tf.keras.Model(inputs, outputs)

In [18]:
model.compile(loss= 'categorical_crossentropy',
              optimizer= tf.keras.optimizers.Adam(),
              metrics= ['accuracy'])

In [ ]:
model.fit(train_data,
          epochs= 10,
          steps_per_epoch= len(train_data),
          validation_data= test_data,
          validation_steps= int(0.15 * len(test_data)))